## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,35.26,93,100,0.00,light intensity drizzle
1,1,Vaini,TO,-21.2000,-175.2000,80.76,89,75,12.66,broken clouds
2,2,Butaritari,KI,3.0707,172.7902,81.73,76,53,15.01,broken clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,33.91,100,20,2.30,few clouds
4,4,Novaya Mayna,RU,54.1500,49.7500,60.22,91,0,5.66,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))

max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 20
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,35.26,93,100,0.00,light intensity drizzle
3,3,Punta Arenas,CL,-53.1500,-70.9167,33.91,100,20,2.30,few clouds
4,4,Novaya Mayna,RU,54.1500,49.7500,60.22,91,0,5.66,clear sky
5,5,Kiunga,PG,-6.1219,141.2906,77.67,90,100,2.48,overcast clouds
6,6,Cape Town,ZA,-33.9258,18.4232,57.38,87,75,5.75,broken clouds
7,7,Ratlam,IN,23.3167,75.0667,76.84,85,100,7.94,overcast clouds
9,9,Port Hardy,CA,50.6996,-127.4199,57.38,94,100,8.05,light rain
10,10,Tasiilaq,GL,65.6145,-37.6368,42.96,60,26,1.95,scattered clouds
11,11,Rikitea,PF,-23.1203,-134.9692,72.57,73,46,20.89,scattered clouds
12,12,Synya,RU,65.3718,58.0387,62.92,53,100,6.96,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()


Unnamed: 0             508
City                   508
Country                502
Lat                    508
Lng                    508
Max Temp               508
Humidity               508
Cloudiness             508
Wind Speed             508
Current Description    508
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

preferred_cities_clean_df = preferred_cities_df.dropna(how="any")

preferred_cities_clean_df.count()


Unnamed: 0             502
City                   502
Country                502
Lat                    502
Lng                    502
Max Temp               502
Humidity               502
Cloudiness             502
Wind Speed             502
Current Description    502
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"

hotel_df["Hotel Name"] = ""

hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ushuaia,AR,35.26,light intensity drizzle,-54.8000,-68.3000,
3,Punta Arenas,CL,33.91,few clouds,-53.1500,-70.9167,
4,Novaya Mayna,RU,60.22,clear sky,54.1500,49.7500,
5,Kiunga,PG,77.67,overcast clouds,-6.1219,141.2906,
6,Cape Town,ZA,57.38,broken clouds,-33.9258,18.4232,
7,Ratlam,IN,76.84,overcast clouds,23.3167,75.0667,
9,Port Hardy,CA,57.38,light rain,50.6996,-127.4199,
10,Tasiilaq,GL,42.96,scattered clouds,65.6145,-37.6368,
11,Rikitea,PF,72.57,scattered clouds,-23.1203,-134.9692,
12,Synya,RU,62.92,overcast clouds,65.3718,58.0387,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    lng = row["Lng"]
    
     #Add the latitude and longitude to location key for the params dictionary.
        
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
        
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
    
        print("Hotel not found... skipping.")

#Indicate that data loading is complete.

print ("Data Retrieval Complete")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [11]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df=hotel_df.dropna(how="any")

clean_hotel_df.head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ushuaia,AR,35.26,light intensity drizzle,-54.8000,-68.3000,Albatross Hotel
3,Punta Arenas,CL,33.91,few clouds,-53.1500,-70.9167,Hotel Hain
4,Novaya Mayna,RU,60.22,clear sky,54.1500,49.7500,
5,Kiunga,PG,77.67,overcast clouds,-6.1219,141.2906,Airport Lodge Kiunga
6,Cape Town,ZA,57.38,broken clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"


In [13]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv

clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>

<dt>Hotel Name</dt><dd>{Hotel Name}</dd>

<dt>City</dt><dd>{City}</dd>

<dt>Country</dt><dd>{Country}</dd>

<dt>Weather Description and </dt><dd>{Current Description}</dd>

<dt>Max Temp</dt><dd>{Max Temp} °F</dd>

</dl>

"""



# 10a. Get the data from each row and add it to the formatting template and store the data in a list.

hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.

locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure

fig = gmaps.figure (center=(30.0, 31.0), zoom_level=1.4)

fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))